In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px

In [2]:
# Read the launch data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                    ],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0, max=10000, step=500,
                                    marks={0: '0', 
                                           2500: '2500',
                                           5000: '5000',
                                           7500: '7500',
                                           10000: '10000'},
                                    value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id="payload-slider", component_property="value")])

def get_pie_chart(entered_site, slider_payload):
    min_slider = slider_payload[0]
    max_slider = slider_payload[1]
    if entered_site == 'ALL':
        filtered_df_all = spacex_df[(spacex_df["Payload Mass (kg)"] >= min_slider) & (spacex_df["Payload Mass (kg)"] <= max_slider)]
        fig = px.pie(filtered_df_all, values='class',
                     names=filtered_df_all['Launch Site'],
                     title='Total Success Launches by Site')
        return fig
    else:
        filtered_df_site = spacex_df[(spacex_df["Launch Site"] == entered_site) & 
                                     ((spacex_df["Payload Mass (kg)"] >= min_slider) & 
                                      (spacex_df["Payload Mass (kg)"] <= max_slider))].groupby('class')['class'].count()
        fig = px.pie(filtered_df_site, values='class',
                     names=filtered_df_site.index,
                     title='Total Success Launches for Site {}'.format(entered_site))
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id="payload-slider", component_property="value")])

def get_scatter_plot(entered_site, slider_payload):
    min_slider = slider_payload[0]
    max_slider = slider_payload[1]
    if entered_site == 'ALL':
        filtered_df_all = spacex_df[(spacex_df["Payload Mass (kg)"] >= min_slider) & (spacex_df["Payload Mass (kg)"] <= max_slider)]
        fig = px.scatter(filtered_df_all, x="Payload Mass (kg)", y="class", 
                         title='Correlation Between Payload and Success for All Sites',
                         color="Booster Version Category")
        return fig
    else:
        filtered_df_site = spacex_df[(spacex_df["Launch Site"] == entered_site) & 
                                     ((spacex_df["Payload Mass (kg)"] >= min_slider) & (spacex_df["Payload Mass (kg)"] <= max_slider))]
        fig = px.scatter(filtered_df_site, x="Payload Mass (kg)", y="class", 
                         title='Correlation between Payload and Success for Site {}'.format(entered_site),
                         color="Booster Version Category")
        return fig
# Run the app
if __name__ == '__main__':
    app.run_server()
